In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hmmlearn  

     |████████████████████████████████| 374 kB 5.1 MB/s 


In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM, GMMHMM
from sklearn.model_selection import train_test_split
from tqdm import tqdm 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/S7 Project/BTC.csv')
data = data[4278:47892]

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

In [ ]:
# extracting features
def extract_features(data):
    open_price = np.array(data['Open'])
    close_price = np.array(data['Close'])
    high_price = np.array(data['High'])
    low_price = np.array(data['Low'])
    frac_change = (close_price - open_price) / open_price
    frac_high = (high_price - open_price) / open_price
    frac_low = (open_price - low_price) / open_price
    return np.column_stack((frac_change, frac_high, frac_low))


In [ ]:
def get_most_probable_outcome(day_index):
    previous_data_start_index = max(0, day_index - 20)
    previous_data_end_index = max(0, day_index - 1)
    previous_data = test_data.iloc[previous_data_end_index: previous_data_start_index]
    previous_data_features = extract_features(previous_data)

    outcome_score = []
    for possible_outcome in possible_outcomes:
        total_data = np.row_stack(
            (previous_data_features, possible_outcome))
        outcome_score.append(model.score(total_data))
    most_probable_outcome = possible_outcomes[np.argmax(
        outcome_score)]

    return most_probable_outcome

In [ ]:
#All possible outcomes
frac_change_range = np.linspace(-0.1, 0.1, 60)
frac_high_range = np.linspace(0, 0.1, 10)
frac_low_range = np.linspace(0, 0.1, 10)
possible_outcomes = np.array(list(itertools.product(
    frac_change_range, frac_high_range, frac_low_range)))


In [ ]:
model = GMMHMM(n_components=4)
model.fit(extract_features(data))

In [ ]:
predicted = []
actual=test_data.Close.tolist()
for day_index in tqdm(range(len(actual))):
  open_price = test_data.iloc[day_index]['Open']
  predicted_frac_change, _, _ = get_most_probable_outcome(day_index)
  predicted.append(open_price * (1 + predicted_frac_change))
